In [1]:
%matplotlib inline

In [2]:
import numpy as np
from scipy.fftpack import fft,ifft
from schrodinger import Schrodinger
import matplotlib.pyplot as plt
import scipy
import scipy.integrate as integrate
import sympy as sp
import ipywidgets as widgets

from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='jshtml')
import seaborn as sns

#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('poster', font_scale=0.9,  rc={"lines.linewidth": 5})

In [3]:
# specify constants
hbar = 1.0   # planck's constant
m = 1.9      # particle mass

### Plot wave-function in space and momentum space

In [68]:
#Specify initial wave function at grid points:
@np.vectorize
def psiF(x,sigma=1.0,k0=1):
    norm = 1/(np.pi*sigma**2)**(1/4)
    f = np.exp(-(x)**2/(4*sigma**2))
    f *= np.exp(complex(0,k0*x))
    
    return  norm*f


@np.vectorize
def psi2F(x,sigma=1.0,k0=1,t=0):
    at = (sigma**4 + hbar**2*t**2/m**2)**(1/4)
    v0 = hbar*k0/m
    
    norm = sigma/(np.sqrt(np.pi)*at**2)
    f = np.exp(-(x-v0*t)**2/(sigma**2*(1+hbar**2*t**2/(m**2*sigma**2))))    
    return  norm*f

@np.vectorize
def J(x,sigma=1.0,k0=1,t=0):
    at = (sigma**4 + hbar**2*t**2/m**2)**(1/4)
    v0 = hbar*k0/m
    
    norm = (hbar/m)*psi2F(x,sigma,k0,t)
        
    f = k0 + (hbar*t/m)*(x-v0*t)/(at**4)
    return  norm*f


@np.vectorize
def dP(x0,dx,sigma,k0,t):
    
    dp = integrate.quad(psi2F, x0-dx/2, x0+dx/2, args=(sigma,k0,t))[0]
    
    return dp


### Animation:

In [80]:
# Set up plot
fig,ax1 = plt.subplots(nrows=1,ncols=1,figsize=(10,8))

dxProb = 5.0
x0 = 0.0
sigmav=0.5
k0v=1.5
dt = 0.1
#define x grid:
x = np.linspace(-20,20,2000)

psi2 = psi2F(x,sigma=sigmav,k0=k0v,t=0.0)

psi_x_line, = ax1.plot([], [])
_,_,bar_container = ax1.hist([x0],bins=[x0-dxProb,x0+dxProb],weights=[dP(x0,dxProb,sigmav,k0v,0.)],alpha=0.7,
                             label=r'$P_V$')
J1 = 1e6*J(x0-dxProb,sigma=sigmav,k0=k0v,t=0)
J2 = 1e6*J(x0+dxProb,sigma=sigmav,k0=k0v,t=0)
ax1.arrow(x0-dxProb,0.5,10*J1,0.,head_width=0.05, head_length=0.4, fc='red', ec='red',label=r'$\vec{J}$')
ax1.arrow(x0+dxProb,0.5,10*J2,0.,head_width=0.05, head_length=0.4, fc='red', ec='red')
ax1.set_xlabel('$x$')
ax1.set_ylabel(r'$|\psi(x)|^2$')
title = ax1.set_title("")
ax1.set_xlim(x.min(),x.max())
ax1.set_ylim(0,max(psi2))
ax1.legend()


# Animate plot
def init():
    psi_x_line.set_data([], [])
    title.set_text("")
    return (psi_x_line, title)

def animate(i):
    t = i*dt
    psi2 = psi2F(x,sigma=sigmav,k0=k0v,t=t)
    psi_x_line.set_data(x, psi2)
    
    P = dP(x0,dxProb,sigmav,k0v,t)
    for rect in bar_container.patches:
        rect.set_height(P)

    J1 = 1e1*J(x0-dxProb,sigma=sigmav,k0=k0v,t=t)
    J2 = 1e1*J(x0+dxProb,sigma=sigmav,k0=k0v,t=t)
    ax1.patches.pop()
    ax1.patches.pop()
    ax1.arrow(x0-dxProb,0.5,10*J1,0.,head_width=0.05, head_length=0.4, fc='red', ec='red')
    ax1.arrow(x0+dxProb,0.5,10*J2,0.,head_width=0.05, head_length=0.4, fc='red', ec='red')

    title.set_text(r"t = %.2f, $P_V$ = %1.1e" % (t,P))
        
    return (psi_x_line, title)

# call the animator.  blit=True means only re-draw the parts that have changed.
N_steps = 50
t_max = 500
frames = int(t_max / float(N_steps * dt))

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=frames, interval=100, blit=True)

#Para salvar corretamente, tem que descomentar a linha de baixo
anim.save('waveCurrent.mp4', fps=15, extra_args=['-vcodec', 'libx264'])

#Para ver o video no notebook deve comentar a linha de cima
plt.close()
anim
